In [1]:
!pip install scipy
!pip install sentence_transformers

In [1]:
import torch
import torch.nn as nn
from sentence_transformers import SentenceTransformer
# import math
# from scipy.stats import special_ortho_group
# import matplotlib.pyplot as plt
# import random

# # for AE and Embedder
# import torch.nn.functional as F
# from torch.utils.data import DataLoader, TensorDataset


# from torchvision.datasets import MNIST
# from torchvision import transforms
# from torch.utils.data import Dataset, DataLoader, random_split
from cINN import ConditionalRealNVP
from autoencoder import AutoencoderSimple
from datasets import CustomMNIST
from utils import get_best_device


## Get the training Data

Loading the pretrained autoencoder and pretrained embedder

In [2]:
embedding_model = SentenceTransformer("intfloat/multilingual-e5-small")

ae_path = "./models/ae_100.pth"
autoencoder = AutoencoderSimple()
autoencoder.load_state_dict(torch.load(ae_path, map_location=get_best_device()))

<All keys matched successfully>

In [3]:
from torch.utils.data import DataLoader

train_dataset = CustomMNIST(train=True)

data_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

Replace images with encoded latent space representation and replace labels with embeddings

In [4]:
from tqdm import tqdm

encoded_dataset = []

for batch_imgs, batch_labels in tqdm(data_loader, desc='Encoding'):
    encoded_imgs = autoencoder.encoder(batch_imgs)
    batch_labels = [str(label) for label in batch_labels]
    encoded_lables = embedding_model.encode(batch_labels, convert_to_tensor=True)
    encoded_dataset.extend(zip(encoded_imgs, encoded_lables))

# Convert labels to strings if not already
# encoded_dataset = [(img, str(label)) for img, label in encoded_dataset]

# # Embed labels efficiently using SentenceTransformer
# embedded_labels = embedding_model.encode(list(zip(*encoded_dataset))[1])
# embedded_dataset = list(zip(list(zip(*encoded_dataset))[0], embedded_labels))

# You can use embedded_dataset for your further processing


KeyboardInterrupt: 

In [8]:
print(len(embedded_dataset))
print(len(embedded_dataset[0]))

60000
2


In [9]:
print(len(emb_model.encode("0")))

384


In [10]:
print(len(embedded_dataset[0][0]))

64


In [11]:
print(len(embedded_dataset[0][1]))

384


In [12]:
cINN = conditional_realNVP(64,128,20,384)

In [13]:
encoded_imgs, embedded_labels = zip(*embedded_dataset)

In [20]:
train_cinn(cINN, encoded_imgs, embedded_labels, batch_size=128)

RuntimeError: Tensors must have same number of dimensions: got 2 and 3